This program will read a csv file of 311 data to a dataframe.
The dataframe will be cleaned as such: Remove all data other than the year 2017

The results will be stored to a new csv file.

In [1]:
import pandas as pd
import numpy as np
import datetime as datetime
import time
import math
%matplotlib inline

from sklearn import preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix

In [2]:
# df311 = pd.read_csv('311cleaned.csv')
# dfwx = pd.read_csv('311weather.csv')
# df = pd.merge(df311, dfwx,  left_on='REQUEST_ID', right_on='REQUEST_ID', how='left')  
# saved merged data to 311Master.csv to save time loading
df = pd.read_csv('311Trimmed.csv')

In [3]:
df.describe()

,REQUEST_ID,STATUS,COUNCIL_DISTRICT,WARD,TRACT,PUBLIC_WORKS_DIVISION,PLI_DIVISION,POLICE_ZONE,X,Y,...,daily.temperatureMin,daily.temperatureMinTime,daily.time,daily.uvIndex,daily.uvIndexTime,daily.visibility,daily.windBearing,daily.windGust,daily.windGustTime,daily.windSpeed
count,49391.000000,49391.000000,49013.000000,48970.000000,4.905500e+04,48974.000000,48970.000000,48946.000000,49391.000000,49391.000000,...,49055.000000,4.905500e+04,4.905500e+04,49055.000000,4.905500e+04,49055.000000,49055.000000,49055.000000,4.905500e+04,49055.000000
mean,163497.231439,1.211456,4.899476,16.630325,4.200323e+10,3.017581,16.630325,3.594594,-79.428335,40.165465,...,48.206375,1.497977e+09,1.497940e+09,5.261767,1.497984e+09,9.372241,216.032555,9.282200,1.497989e+09,1.536781
std,23536.455461,0.712449,2.588323,7.928818,1.811907e+05,1.432717,7.928818,1.605884,6.573827,3.324300,...,15.380325,8.201252e+06,8.201741e+06,2.416315,8.201483e+06,0.871285,73.600925,4.524205,8.203111e+06,1.162964
min,123272.000000,0.000000,1.000000,0.000000,4.200301e+10,0.000000,0.000000,1.000000,-80.088105,0.000000,...,1.200000,1.483276e+09,1.483247e+09,1.000000,1.483290e+09,3.780000,0.000000,1.460000,1.483247e+09,0.020000
25%,143202.500000,1.000000,3.000000,11.000000,4.200311e+10,2.000000,11.000000,2.000000,-80.007489,40.420436,...,37.170000,1.491361e+09,1.491278e+09,3.000000,1.491322e+09,9.050000,188.000000,5.730000,1.491332e+09,0.710000
50%,163258.000000,1.000000,5.000000,16.000000,4.200318e+10,3.000000,16.000000,4.000000,-79.976893,40.445355,...,51.120000,1.497928e+09,1.497845e+09,5.000000,1.497895e+09,9.750000,224.000000,8.810000,1.497902e+09,1.310000
75%,183529.500000,1.000000,7.000000,22.000000,4.200329e+10,5.000000,22.000000,5.000000,-79.934224,40.462244,...,60.880000,1.504091e+09,1.504066e+09,7.000000,1.504109e+09,10.000000,267.000000,11.620000,1.504112e+09,2.110000
max,205290.000000,3.000000,9.000000,32.000000,4.200398e+10,6.000000,32.000000,6.000000,0.000000,40.520177,...,73.950000,1.514722e+09,1.514696e+09,10.000000,1.514732e+09,10.000000,359.000000,26.100000,1.514696e+09,6.150000


In [3]:
df.fillna(0, inplace=True) #fill zero instances of weather
df = df.reset_index()

In [5]:
df.head()

,index,REQUEST_ID,CREATED_ON,REQUEST_TYPE,REQUEST_ORIGIN,STATUS,DEPARTMENT,NEIGHBORHOOD,COUNCIL_DISTRICT,WARD,...,daily.temperatureMin,daily.temperatureMinTime,daily.time,daily.uvIndex,daily.uvIndexTime,daily.visibility,daily.windBearing,daily.windGust,daily.windGustTime,daily.windSpeed
0,0,205290,2017-12-31T23:17:00,Snow/Ice removal,Website,1,DPW - Street Maintenance,Marshall-Shadeland,1.0,27.0,...,1.99,1.514722e+09,1.514696e+09,1.0,1.514732e+09,9.09,264.0,5.32,1.514696e+09,0.57
1,1,205289,2017-12-31T22:01:00,Snow/Ice removal,Report2Gov iOS,1,DPW - Street Maintenance,Stanton Heights,9.0,10.0,...,1.57,1.514722e+09,1.514696e+09,1.0,1.514732e+09,9.06,265.0,5.27,1.514696e+09,0.51
2,2,205288,2017-12-31T21:55:00,Snow/Ice removal,Website,1,DPW - Street Maintenance,New Homestead,5.0,31.0,...,1.76,1.514722e+09,1.514696e+09,1.0,1.514732e+09,9.06,257.0,5.38,1.514696e+09,0.63
3,3,205287,2017-12-31T18:23:00,Missed Pick Up,Website,1,DPW - Refuse,Mount Washington,2.0,19.0,...,1.50,1.514722e+09,1.514696e+09,1.0,1.514732e+09,9.09,262.0,5.35,1.514696e+09,0.55
4,4,205286,2017-12-31T17:56:00,Snow/Ice removal,Report2Gov iOS,1,DPW - Street Maintenance,Stanton Heights,9.0,10.0,...,1.57,1.514722e+09,1.514696e+09,1.0,1.514732e+09,9.06,265.0,5.27,1.514696e+09,0.52


In [6]:
df.columns

Index(['index', 'REQUEST_ID', 'CREATED_ON', 'REQUEST_TYPE', 'REQUEST_ORIGIN',
       'STATUS', 'DEPARTMENT', 'NEIGHBORHOOD', 'COUNCIL_DISTRICT', 'WARD',
       'TRACT', 'PUBLIC_WORKS_DIVISION', 'PLI_DIVISION', 'POLICE_ZONE',
       'FIRE_ZONE', 'X', 'Y', 'GEO_ACCURACY', 'current.apparentTemperature',
       'current.cloudCover', 'current.dewPoint', 'current.humidity',
       'current.icon', 'current.precipIntensity', 'current.precipProbability',
       'current.precipType', 'current.pressure', 'current.summary',
       'current.temperature', 'current.time', 'current.uvIndex',
       'current.visibility', 'current.windBearing', 'current.windGust',
       'current.windSpeed', 'daily.apparentTemperatureHigh',
       'daily.apparentTemperatureHighTime', 'daily.apparentTemperatureLow',
       'daily.apparentTemperatureLowTime', 'daily.apparentTemperatureMax',
       'daily.apparentTemperatureMaxTime', 'daily.apparentTemperatureMin',
       'daily.apparentTemperatureMinTime', 'daily.cloudCov

In [ ]:
df['REQUEST_TYPE'].value_counts(sort=True, ascending=False, dropna=False) 

In [ ]:
df['REQUEST_TYPE'].value_counts(sort=True, ascending=False, dropna=False).hist(bins=50)

In [ ]:
df['REQUEST_TYPE'].value_counts(sort=True, ascending=False, dropna=False) 

In [ ]:
df['REQUEST_TYPE'].value_counts(sort=True, ascending=False, dropna=False).plot.barh()

In [ ]:
df['REQUEST_ORIGIN'].value_counts(sort=True, ascending=False, dropna=False)

In [ ]:
# visualizatins of various fields and weather events
# time is based off the unix timestamp, this graph covers the entire year
df['daily.time'].hist(bins=50)

In [ ]:
df['current.precipIntensity'].hist()

In [ ]:
df['daily.precipType'].value_counts(sort=True, ascending=False, dropna=False).plot.bar()

In [ ]:
df['current.visibility'].hist(bins=10)

In [ ]:
df['current.pressure'].hist(bins=100)

In [ ]:
df['daily.pressure'].hist(bins=100)

In [ ]:
df['daily.windSpeed'].hist(bins=1000)

In [ ]:
df['daily.windGust'].hist(bins=1000)

In [ ]:
df['daily.precipProbability'].hist(bins=100)

In [ ]:
df['current.summary'].value_counts(sort=True, ascending=False, dropna=False)

In [ ]:
df['current.summary'].value_counts(sort=True, ascending=False, dropna=False).plot.bar()

In [ ]:
df['daily.temperatureHigh'].hist(bins=100)

In [ ]:
df['daily.temperatureLow'].hist(bins=100)

In [ ]:
df['daily.precipProbability'].hist(bins=100)

In [ ]:
df['daily.precipAccumulation'].hist(bins=50)

In [ ]:
df['daily.humidity'].hist(bins=100)

In [ ]:
df['daily.pressure'].hist(bins=100)

In [4]:
df['REQUEST_TYPE'].value_counts(sort=True, ascending=False, dropna=False) 

Potholes                                8689
Weeds/Debris                            7873
Building Maintenance                    4037
Abandoned Vehicle (parked on street)    2572
Litter                                  1886
Refuse Violations                       1802
Illegal Parking                         1770
Missed Pick Up                          1717
Replace/Repair a Sign                   1539
City Source (CDBG)                      1499
Street Light - Repair                   1422
Building Without a Permit               1348
Vacant Building                         1256
Paving Request                          1236
Overgrowth                              1209
Snow/Ice removal                        1033
Pruning (city tree)                     1020
Rodent control                           965
Need Potable Water                       845
Fire Safety System Not Working           818
Dead Animal                              787
Request New Sign                         783
Illegal Du

In [5]:
df['DEPARTMENT'].value_counts(sort=True, ascending=False, dropna=False) 

Permits, Licenses and Inspections       15332
DPW - Street Maintenance                12596
DPW - Refuse                             3519
Police - AVU                             2572
Pittsburgh Water and Sewer Authority     2266
Police - Zones 1-6                       2089
Animal Care & Control                    1752
DOMI - TrafficShop                       1539
City Source Associates, Inc.             1499
Allegheny City Electric                  1422
DOMI - Asphalt                           1236
DOMI - Traffic                           1187
DPW - Forestry Division                  1020
DOMI - Permits                            756
DOMI - Streets                            146
DPW - Park Maintenance                     75
311                                        46
0                                           2
DPW - Transportation & Engineering          1
Name: DEPARTMENT, dtype: int64

In [6]:
df['NEIGHBORHOOD'].value_counts(sort=True, ascending=False, dropna=False) 

Brookline                    2211
Carrick                      2145
South Side Slopes            1799
Squirrel Hill South          1410
South Side Flats             1397
Mount Washington             1330
Beechview                    1236
Hazelwood                    1218
Bloomfield                   1178
Lincoln-Lemington-Belmar     1165
Knoxville                    1056
Central Lawrenceville        1049
Homewood North                998
Greenfield                    998
Garfield                      968
East Liberty                  959
Shadyside                     871
Marshall-Shadeland            866
Homewood South                841
Sheraden                      830
Elliott                       816
Brighton Heights              811
Stanton Heights               764
Perry South                   722
Middle Hill                   721
Perry North                   709
Central Business District     695
Overbrook                     689
Highland Park                 665
Allentown     

In [6]:
request_list = ['Potholes', 'Weeds/Debris', 'Litter', 'Paving Request']

In [7]:
df_holes = df[df['REQUEST_TYPE'] == 'Potholes']
df_weeds = df[df['REQUEST_TYPE'] == 'Weeds/Debris']
df_abandoned = df[df['REQUEST_TYPE'] == 'Abandoned Vehicle (parked on street)']
df_litter = df[df['REQUEST_TYPE'] == 'Litter']
df_paving = df[df['REQUEST_TYPE'] == 'Paving Request']





In [ ]:
df_holes.describe()

In [ ]:
df_weeds.describe()

# Feature Selection

In [8]:
def label_encode(df, columns):
    for col in columns:
        le = prep.LabelEncoder()
        col_values_unique = list(df[col].unique())
        le_fitted = le.fit(col_values_unique)
 
        col_values = list(df[col].values)
        #le.classes_
        col_values_transformed = le.transform(col_values)
        df[col] = col_values_transformed

In [ ]:
X = df[['daily.apparentTemperatureHigh', 'daily.apparentTemperatureLow',
       'daily.apparentTemperatureMax',
       'daily.apparentTemperatureMin',
       'daily.cloudCover',
       'daily.dewPoint', 'daily.humidity',
       'daily.precipAccumulation', 'daily.precipIntensity',
       'daily.precipIntensityMax', 
       'daily.precipProbability', 'daily.pressure',
       'daily.temperatureHigh', 
       'daily.temperatureLow', 
       'daily.temperatureMax', 
       'daily.temperatureMin', 
       'daily.uvIndex', 'daily.visibility',
       'daily.windBearing', 'daily.windGust', 
       'daily.windSpeed']]
y = df[['REQUEST_TYPE']]

In [21]:
# as features/target
#all features excluding times
features_all = df[['current.apparentTemperature', 'current.cloudCover', 'current.dewPoint', 'current.humidity',
       'current.icon', 'current.precipIntensity', 'current.precipProbability',
       'current.precipType', 'current.pressure', 'current.summary',
       'current.temperature', 'current.uvIndex',
       'current.visibility', 'current.windBearing', 'current.windGust',
       'current.windSpeed', 'daily.apparentTemperatureHigh',
       'daily.apparentTemperatureLow', 'daily.apparentTemperatureMax',
       'daily.apparentTemperatureMin', 'daily.cloudCover',
       'daily.dewPoint', 'daily.humidity', 'daily.icon', 'daily.moonPhase',
       'daily.precipAccumulation', 'daily.precipIntensity', 'daily.precipIntensityMax', 
       'daily.precipProbability', 'daily.precipType', 'daily.pressure',
       'daily.summary', 'daily.temperatureHigh', 'daily.temperatureLow', 
       'daily.temperatureMax', 'daily.temperatureMin', 'daily.uvIndex', 'daily.visibility',
       'daily.windBearing', 'daily.windGust', 'daily.windSpeed']]

#most highly correlated features
features = df[['current.cloudCover', 'current.humidity',
       'current.icon', 'current.precipProbability',
       'current.precipType', 'current.summary',
       'current.temperature', 'current.uvIndex',
       'daily.apparentTemperatureMin',
       'daily.cloudCover', 'daily.dewPoint', 'daily.humidity',
       'daily.precipAccumulation', 'daily.precipIntensity',
       'daily.precipProbability', 'daily.precipType',  
       'daily.temperatureMin', 'daily.uvIndex', 'daily.windSpeed']]
       
label_encode(features_all, features_all.columns.values)
label_encode(features, features.columns.values)

target = df[['REQUEST_TYPE']]
label_encode(target, target.columns.values)

C:\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


### Correlations

In [22]:
X_train, X_test, y_train, y_test = train_test_split(features_all, target, test_size = 0.2, random_state = 1)
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_model.coef_[0][idx]))

The coefficient for current.apparentTemperature is 0.0008093613863588357
The coefficient for current.cloudCover is -0.0037119627895202474
The coefficient for current.dewPoint is 1.3409912479605674e-05
The coefficient for current.humidity is -0.0021480800344471454
The coefficient for current.icon is 0.02403535125486922
The coefficient for current.precipIntensity is 0.0012802547149778313
The coefficient for current.precipProbability is -0.011554449934477594
The coefficient for current.precipType is 0.025345539228364418
The coefficient for current.pressure is -0.00018096995981898594
The coefficient for current.summary is -0.010728954833986162
The coefficient for current.temperature is -0.0010486497248902876
The coefficient for current.uvIndex is 0.009329663145923494
The coefficient for current.visibility is 0.0001577448749850137
The coefficient for current.windBearing is -4.233677063335327e-05
The coefficient for current.windGust is 7.105487522226451e-05
The coefficient for current.windSp

In [24]:
#top features
for idx, col_name in enumerate(X_train.columns):
    if abs(regression_model.coef_[0][idx]) > 0.002:
        print("The coefficient for {} is {}".format(col_name, regression_model.coef_[0][idx]))

The coefficient for current.cloudCover is -0.0037119627895202474
The coefficient for current.humidity is -0.0021480800344471454
The coefficient for current.icon is 0.02403535125486922
The coefficient for current.precipProbability is -0.011554449934477594
The coefficient for current.precipType is 0.025345539228364418
The coefficient for current.summary is -0.010728954833986162
The coefficient for current.uvIndex is 0.009329663145923494
The coefficient for daily.apparentTemperatureMin is -0.00400191062739003
The coefficient for daily.cloudCover is -0.0034682516175310813
The coefficient for daily.dewPoint is 0.003176535276032792
The coefficient for daily.humidity is -0.0661964776021008
The coefficient for daily.icon is -0.05422648899691992
The coefficient for daily.precipAccumulation is -0.004671375214388841
The coefficient for daily.precipProbability is 0.0030214432443727147
The coefficient for daily.precipType is 0.022543721307223163
The coefficient for daily.temperatureMin is 0.0035458

# Linear Regression

In [25]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 1)
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [27]:
y_pred = regression_model.predict(X_test)
rmse = mean_squared_error(y_pred, y_test)
regression_model.score(X_test, y_test),  rmse, math.sqrt(rmse)

(0.006733526609910446, 65.86204639886435, 8.115543506066883)

In [28]:
confusion = confusion_matrix(y_test, y_pred.round())
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

sensitivity = TP / float(FN + TP)
specificity = TN / (TN + FP)
false_positive_rate = FP / float(TN + FP)
precision = TP / float(TP + FP)

print ("sensitivity: " + str(sensitivity) + 
       "\nspecificity: " + str(specificity) + 
       "\nfalse positive rate: " + str(false_positive_rate) + 
       "\nprecision: " + str(precision))

sensitivity: nan
specificity: nan
false positive rate: nan
precision: nan


C:\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
C:\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  
C:\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
C:\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


# NB

In [29]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 1)
nb = GaussianNB()
nb.fit(X_train, y_train)

C:\dev\Anaconda3\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None, var_smoothing=1e-09)

In [30]:
y_pred = nb.predict(X_test)
y_pred

array([13, 12, 13, ..., 13,  4, 25], dtype=int64)

In [31]:
accuracy_score(y_test, y_pred, normalize=True)

0.12197590849276242

In [32]:
confusion = confusion_matrix(y_test, y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

sensitivity = TP / float(FN + TP)
specificity = TN / (TN + FP)
false_positive_rate = FP / float(TN + FP)
precision = TP / float(TP + FP)

print ("sensitivity: " + str(sensitivity) + 
       "\nspecificity: " + str(specificity) + 
       "\nfalse positive rate: " + str(false_positive_rate) + 
       "\nprecision: " + str(precision))

sensitivity: nan
specificity: nan
false positive rate: nan
precision: nan


C:\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
C:\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  
C:\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
C:\dev\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


# KNN

In [33]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 1)
model = KNeighborsRegressor(n_neighbors=4)
model.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=4, p=2,
          weights='uniform')

In [34]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred.round(), normalize=False) / len(y_test.index)

0.0792590343152141

In [35]:
confusion = confusion_matrix(y_test, y_pred.round())
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

sensitivity = TP / float(FN + TP)
specificity = TN / (TN + FP)
false_positive_rate = FP / float(TN + FP)
precision = TP / float(TP + FP)

print ("sensitivity: " + str(sensitivity) + 
       "\nspecificity: " + str(specificity) + 
       "\nfalse positive rate: " + str(false_positive_rate) + 
       "\nprecision: " + str(precision))

sensitivity: 0.9655172413793104
specificity: 0.75
false positive rate: 0.25
precision: 0.9655172413793104


# SVM (Runs forever)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 1)
model = svm.SVC() 
model.fit(X_train, y_train)

C:\dev\Anaconda3\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\dev\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
confusion = metrics.confusion_matrix(y_test, y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

sensitivity = TP / float(FN + TP)
specificity = TN / (TN + FP)
false_positive_rate = FP / float(TN + FP)
precision = TP / float(TP + FP)

print ("sensitivity: " + str(sensitivity) + 
       "\nspecificity: " + str(specificity) + 
       "\nfalse positive rate: " + str(false_positive_rate) + 
       "\nprecision: " + str(precision))

# Random Forest